In [1]:
import pandas as pd
from apyori import apriori
from sklearn.compose import make_column_selector #Para seleccionar las variables por tipo
from sklearn.preprocessing import OneHotEncoder, KBinsDiscretizer #Los transformadores que se usaran
from sklearn.compose import ColumnTransformer #El preprocesador

In [2]:
mtmovies = pd.read_csv('movies.csv', encoding = "ISO-8859-1")
mtmovies.head()

,id,budget,genres,homePage,productionCompany,productionCompanyCountry,productionCountry,revenue,runtime,video,...,popularity,releaseDate,voteAvg,voteCount,genresAmount,productionCoAmount,productionCountriesAmount,actorsAmount,castWomenAmount,castMenAmount
0,5,4000000,Crime|Comedy,https://www.miramax.com/movie/four-rooms/,Miramax|A Band Apart,US|US,United States of America,4257354.0,98,False,...,20.880,1995-12-09,5.7,2077,2,2,1,25,15,9
1,6,21000000,Action|Thriller|Crime,NaN,Universal Pictures|Largo Entertainment|JVC,US|US|JP,Japan|United States of America,12136938.0,110,False,...,9.596,1993-10-15,6.5,223,3,3,2,15,3,9
2,11,11000000,Adventure|Action|Science Fiction,http://www.starwars.com/films/star-wars-episod...,Lucasfilm|20th Century Fox,US|US,United States of America,775398007.0,121,NaN,...,100.003,1977-05-25,8.2,16598,3,2,1,105,5,62
3,12,94000000,Animation|Family,http://movies.disney.com/finding-nemo,Pixar,US,United States of America,940335536.0,100,NaN,...,134.435,2003-05-30,7.8,15928,2,1,1,24,5,18
4,13,55000000,Comedy|Drama|Romance,NaN,Paramount|The Steve Tisch Company,US|,United States of America,677387716.0,142,False,...,58.751,1994-07-06,8.5,22045,3,2,1,76,18,48


In [3]:
print(mtmovies.shape)
print(mtmovies['genres'].value_counts())

(10000, 27)
genres
Drama                                          521
Comedy                                         440
Horror                                         230
Drama|Romance                                  211
Horror|Thriller                                205
                                              ... 
Music|Romance                                    1
Adventure|Animation|Fantasy|TV Movie|Comedy      1
Horror|Mystery|Family                            1
Action|Thriller|Drama|Adventure                  1
Western|Drama|Mystery                            1
Name: count, Length: 2345, dtype: int64


In [68]:
categoricas = mtmovies[['genres', 'homePage', 'productionCompany', 'productionCompanyCountry', 'video', 'director', 'actors', 'actorsPopularity', 'actorsCharacter', 'originalTitle', 'title', 'originalLanguage', 'releaseDate', 'castWomenAmount', 'castMenAmount']]
categoricas = categoricas.astype(object)
categoricas['genres'].unique()
categoricas['homePage'].unique()
categoricas['productionCompany'].unique()
categoricas['video'].unique()
categoricas['director'].unique()
categoricas['actors'].unique()
categoricas['actorsPopularity'].unique()
categoricas['actorsCharacter'].unique()
categoricas['originalTitle'].unique()
categoricas['title'].unique()
categoricas['originalLanguage'].unique()
categoricas['releaseDate'].unique()
categoricas['castWomenAmount'].unique()
categoricas['castMenAmount'].unique()
categoricas

,genres,homePage,productionCompany,productionCompanyCountry,video,director,actors,actorsPopularity,actorsCharacter,originalTitle,title,originalLanguage,releaseDate,castWomenAmount,castMenAmount
0,Crime|Comedy,https://www.miramax.com/movie/four-rooms/,Miramax|A Band Apart,US|US,False,Allison Anders|Alexandre Rockwell|Robert Rodri...,Tim Roth|Jennifer Beals|Antonio Banderas|Valer...,22.225|23.519|17.816|19.893|9.027|7.147|7.769|...,Ted the Bellhop|Angela|Man|Athena|Siegfried|Je...,Four Rooms,Four Rooms,en,1995-12-09,15,9
1,Action|Thriller|Crime,NaN,Universal Pictures|Largo Entertainment|JVC,US|US|JP,False,Stephen Hopkins,Emilio Estevez|Cuba Gooding Jr.|Denis Leary|St...,9.008|6.383|10.757|18.295|11.772|14.777|9.669|...,Frank Wyatt|Mike Peterson|Fallon|John Wyatt|Ra...,Judgment Night,Judgment Night,en,1993-10-15,3,9
2,Adventure|Action|Science Fiction,http://www.starwars.com/films/star-wars-episod...,Lucasfilm|20th Century Fox,US|US,NaN,George Lucas,FALSE,11.881|24.542|14.434|10.651|6.888|5.811|1.432|...,Luke Skywalker|Han Solo|Princess Leia Organa|G...,Star Wars,Star Wars,en,1977-05-25,5,62
3,Animation|Family,http://movies.disney.com/finding-nemo,Pixar,US,NaN,Andrew Stanton,FALSE,9.79|8.084|8.538|33.379|11.733|11.866|8.141|12...,Marlin (voice)|Dory (voice)|Nemo (voice)|Gill ...,Finding Nemo,Finding Nemo,en,2003-05-30,5,18
4,Comedy|Drama|Romance,NaN,Paramount|The Steve Tisch Company,US|,False,Robert Zemeckis,Tom Hanks|Robin Wright|Gary Sinise|Mykelti Wil...,33.297|13.246|6.748|6.402|13.189|10.974|10.259...,Forrest Gump|Jenny Curran|Dan Taylor|Bubba|Mrs...,Forrest Gump,Forrest Gump,en,1994-07-06,18,48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,Action|Horror,NaN,NaN,NaN,False,Mario N. Bonassin,Eric Roberts|Emilia Torello|Joe Herrera|Michel...,18.185|0.828|0.6|0.6|0.6|1.102|3.069|0.6|0.6|0.6,Dr. Malone|Grace|Joaquin|Allison|Rex|Jason|Rit...,Megaboa,Megaboa,en,2021-11-26,2,4
9996,Comedy,NaN,Caracol Televisión|Dago García Producciones,CO|CO,False,Rodrigo Triana,Amparo Grisales|Jhon Álex Toro|Chichila Navia|...,1.55|0.84|0.6|0.6|0.6|0.6|0.6|0.6,La abuela - Raquel|El papá|La mamá|Sarita|El h...,El paseo 6,El Paseo 6,es,2021-12-25,1,1
9997,Comedy,NaN,NaN,NaN,False,Niyi Akinmolayan,Shaffy Bello-Akinrimisi|Funke Akindele|Joke Si...,0.6|0.98|0.6|0.6|0.6|0.6|0.6|0.6|1.4|0.6|0.6|1...,||||||||||||||||,Chief Daddy 2: Going for Broke,Chief Daddy 2: Going for Broke,en,2021-12-17,0,922017
9998,NaN,https://www.netflix.com/title/81425229,NaN,NaN,False,NaN,NaN,NaN,922162,The Witcher: Fireplace,The Witcher: Fireplace,en,2021-12-17,922162,The Witcher: Fireplace


In [69]:
print(categoricas.shape)
# Se van a convertir los datos a una lista de listas
records = []

for i in range(0, 32):
    records.append([str(categoricas.values[i,j]) for j in range(0, 15)])
records


(10000, 15)


[['Crime|Comedy',
  'https://www.miramax.com/movie/four-rooms/',
  'Miramax|A Band Apart',
  'US|US',
  'False',
  'Allison Anders|Alexandre Rockwell|Robert Rodriguez|Quentin Tarantino',
  'Tim Roth|Jennifer Beals|Antonio Banderas|Valeria Golino|David Proval|Sammi Davis|Amanda de Cadenet|Madonna|Ione Skye|Lili Taylor|Alicia Witt|Tamlyn Tomita|Lana McKissack|Danny Verduzco|Patricia Vonne|Salma Hayek|Quentin Tarantino|Paul Calderon|Bruce Willis|Marisa Tomei|Lawrence Bender|Kathy Griffin|Kimberly Blair|Marc Lawrence|Paul Skemp',
  '22.225|23.519|17.816|19.893|9.027|7.147|7.769|6.476|7.906|12.838|17.988|20.027|4.47|0.6|1.286|23.95|22.082|3.44|27.973|19.062|2.688|2.079|1.008|7.647|0.6',
  'Ted the Bellhop|Angela|Man|Athena|Siegfried|Jezebel|Diana|Elspeth|Eva|Raven|Kiva|Wife|Sarah|Juancho|Corpse|TV Dancing Girl|Chester Rush|Norman|Leo (uncredited)|Margaret|Long Hair Yuppy Scum|Betty|Hooker|Sam the Bellhop|Real Theodore',
  'Four Rooms',
  'Four Rooms',
  'en',
  '1995-12-09',
  '15',
  '9'],

In [70]:
# El mínimo de cobertura o soporte es 20% y el mínimo de confianza es 70%
reglas_asociacion = apriori(records,min_support=0.2, min_confidence = 0.7)
reglas = list(reglas_asociacion)

In [71]:
print(len(reglas))

6


In [72]:
print(reglas[0])

RelationRecord(items=frozenset({'False'}), support=0.9375, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'False'}), confidence=0.9375, lift=1.0)])


In [73]:
list(reglas)

[RelationRecord(items=frozenset({'False'}), support=0.9375, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'False'}), confidence=0.9375, lift=1.0)]),
 RelationRecord(items=frozenset({'en'}), support=0.9375, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'en'}), confidence=0.9375, lift=1.0)]),
 RelationRecord(items=frozenset({'False', 'en'}), support=0.875, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'False', 'en'}), confidence=0.875, lift=1.0), OrderedStatistic(items_base=frozenset({'False'}), items_add=frozenset({'en'}), confidence=0.9333333333333333, lift=0.9955555555555555), OrderedStatistic(items_base=frozenset({'en'}), items_add=frozenset({'False'}), confidence=0.9333333333333333, lift=0.9955555555555555)]),
 RelationRecord(items=frozenset({'False', 'nan'}), support=0.5625, ordered_statistics=[OrderedStatistic(items_base=frozenset({'nan'}), items_add=frozenset({'False'}), co

In [74]:
def inspect(output):
    lhs = []
    rhs = []
    support = []
    confidence = []
    lift = []

    for result in output:
        if result.ordered_statistics:  # Asegurar que hay reglas dentro
            rule = result.ordered_statistics[0]  # Tomar solo la primera regla
            lhs.append(', '.join(rule.items_base))  # Lado izquierdo (LHS)
            rhs.append(', '.join(rule.items_add))  # Lado derecho (RHS)
            support.append(result.support)
            confidence.append(rule.confidence)
            lift.append(rule.lift)

    return list(zip(lhs, rhs, support, confidence, lift))

# Convertir reglas en DataFrame
output_DataFrame = pd.DataFrame(inspect(reglas), columns=['Left_Hand_Side', 'Right_Hand_Side', 'Support', 'Confidence', 'Lift'])

# Mostrar los resultados
output_DataFrame

,Left_Hand_Side,Right_Hand_Side,Support,Confidence,Lift
0,,False,0.9375,0.9375,1.000000
1,,en,0.9375,0.9375,1.000000
2,,"False, en",0.8750,0.8750,1.000000
3,nan,False,0.5625,0.9000,0.960000
4,nan,en,0.5625,0.9000,0.960000
5,nan,"False, en",0.5000,0.8000,0.914286


In [15]:
# El mínimo de cobertura o soporte es 10% y el mínimo de confianza es 80%
reglas_asociacion = apriori(records,min_support=0.1, min_confidence = 0.8)
reglas = list(reglas_asociacion)

print(len(reglas))

29


In [17]:
def inspect(output):
    lhs = []
    rhs = []
    support = []
    confidence = []
    lift = []

    for result in output:
        if result.ordered_statistics:  # Asegurar que hay reglas dentro
            rule = result.ordered_statistics[0]  # Tomar solo la primera regla
            lhs.append(', '.join(rule.items_base))  # Lado izquierdo (LHS)
            rhs.append(', '.join(rule.items_add))  # Lado derecho (RHS)
            support.append(result.support)
            confidence.append(rule.confidence)
            lift.append(rule.lift)

    return list(zip(lhs, rhs, support, confidence, lift))

# Convertir reglas en DataFrame
output_DataFrame = pd.DataFrame(inspect(reglas), columns=['Left_Hand_Side', 'Right_Hand_Side', 'Support', 'Confidence', 'Lift'])

# Mostrar los resultados
output_DataFrame

,Left_Hand_Side,Right_Hand_Side,Support,Confidence,Lift
0,,False,0.93750,0.937500,1.000000
1,,en,0.93750,0.937500,1.000000
2,13,False,0.12500,1.000000,1.066667
3,14,False,0.12500,1.000000,1.066667
4,14,en,0.12500,1.000000,1.066667
5,14,nan,0.12500,1.000000,1.600000
6,4,False,0.12500,1.000000,1.066667
7,4,en,0.12500,1.000000,1.066667
8,US|US,False,0.15625,0.833333,0.888889
9,United States of America,False,0.56250,0.900000,0.960000


In [18]:
# El mínimo de cobertura o soporte es 30% y el mínimo de confianza es 70%
reglas_asociacion = apriori(records,min_support=0.3, min_confidence = 0.7)
reglas = list(reglas_asociacion)

print(len(reglas))

9


In [19]:
def inspect(output):
    lhs = []
    rhs = []
    support = []
    confidence = []
    lift = []

    for result in output:
        if result.ordered_statistics:  # Asegurar que hay reglas dentro
            rule = result.ordered_statistics[0]  # Tomar solo la primera regla
            lhs.append(', '.join(rule.items_base))  # Lado izquierdo (LHS)
            rhs.append(', '.join(rule.items_add))  # Lado derecho (RHS)
            support.append(result.support)
            confidence.append(rule.confidence)
            lift.append(rule.lift)

    return list(zip(lhs, rhs, support, confidence, lift))

# Convertir reglas en DataFrame
output_DataFrame = pd.DataFrame(inspect(reglas), columns=['Left_Hand_Side', 'Right_Hand_Side', 'Support', 'Confidence', 'Lift'])

# Mostrar los resultados
output_DataFrame

,Left_Hand_Side,Right_Hand_Side,Support,Confidence,Lift
0,,False,0.9375,0.9375,1.000000
1,,en,0.9375,0.9375,1.000000
2,United States of America,False,0.5625,0.9000,0.960000
3,,"False, en",0.8750,0.8750,1.000000
4,nan,False,0.5625,0.9000,0.960000
5,United States of America,en,0.6250,1.0000,1.066667
6,nan,en,0.5625,0.9000,0.960000
7,United States of America,"False, en",0.5625,0.9000,1.028571
8,nan,"False, en",0.5000,0.8000,0.914286


In [31]:
import pandas as pd
from apyori import apriori

# Cargar dataset
mtmovies = pd.read_csv('movies.csv', encoding='ISO-8859-1')

# Seleccionar variables categóricas relevantes
categoricas = mtmovies[['genres', 'productionCompany', 'productionCountry', 'director']].astype(str)

# Separar valores de columnas categóricas en listas
for col in categoricas.columns:
    categoricas[col] = categoricas[col].str.split('|')

# Explode para manejar múltiples valores por película, una columna a la vez
for col in categoricas.columns:
    categoricas = categoricas.explode(col)

# Convertir a formato binario (one-hot encoding)
categoricas_encoded = pd.get_dummies(categoricas)

# Eliminar columnas irrelevantes
columns_to_remove = ['False', 'nan', 'en', 'United States of America', 'US|US']
categoricas_encoded = categoricas_encoded.drop(columns=[col for col in categoricas_encoded.columns if any(x in col for x in columns_to_remove)], errors='ignore')

# Crear transacciones en formato lista de listas
records = categoricas_encoded.apply(lambda row: list(row.index[row.astype(bool)]), axis=1).tolist()

# Eliminar listas vacías en records
print(f'Total transacciones antes de limpiar: {len(records)}')
records = [r for r in records if len(r) > 0]
print(f'Total transacciones después de limpiar: {len(records)}')

# Mostrar algunas transacciones para validar el formato
def print_sample_records(records, num_samples=5):
    print('Ejemplo de transacciones:')
    for i, record in enumerate(records[:num_samples]):
        print(f'Transacción {i+1}: {record}')
print_sample_records(records)

# Aplicar Apriori con valores recomendados
min_support = 0.005  # Ajustado para obtener reglas significativas
min_confidence = 0.75
if len(records) > 0:
    print('Ejecutando Apriori con min_support:', min_support, 'y min_confidence:', min_confidence)
    reglas = list(apriori(records, min_support=min_support, min_confidence=min_confidence))
    print(f'Total reglas generadas: {len(reglas)}')
else:
    print('No hay transacciones válidas para Apriori.')
    reglas = []

# Función para extraer reglas en formato DataFrame
def inspect(output):
    lhs, rhs, support, confidence, lift = [], [], [], [], []
    for result in output:
        for rule in result.ordered_statistics:  # Acceder a todas las reglas en cada resultado
            lhs.append(', '.join(rule.items_base))
            rhs.append(', '.join(rule.items_add))
            support.append(result.support)
            confidence.append(rule.confidence)
            lift.append(rule.lift)
    return list(zip(lhs, rhs, support, confidence, lift))

# Convertir a DataFrame
output_DataFrame = pd.DataFrame(inspect(reglas), columns=['Left_Hand_Side', 'Right_Hand_Side', 'Support', 'Confidence', 'Lift'])

# Mostrar resultados
print(output_DataFrame)

Total transacciones antes de limpiar: 148581
Total transacciones después de limpiar: 146254
Ejemplo de transacciones:
Transacción 1: ['productionCompany_Miramax', 'director_Allison Anders']
Transacción 2: ['productionCompany_Miramax', 'director_Alexandre Rockwell']
Transacción 3: ['productionCompany_Miramax', 'director_Robert Rodriguez']
Transacción 4: ['productionCompany_Miramax']
Transacción 5: ['productionCompany_A Band Apart', 'director_Allison Anders']
Ejecutando Apriori con min_support: 0.005 y min_confidence: 0.75
Total reglas generadas: 1
             Left_Hand_Side          Right_Hand_Side   Support  Confidence  \
0  director_Kunihiko Yuyama  productionCountry_Japan  0.005196    0.834248   

        Lift  
0  13.142193  
